In [1]:
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
import matplotlib.pyplot as plt

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

#Scarping URL
url="https://data-class-mars-challenge.s3.amazonaws.com/Mars/index.html"
browser.visit(url)

In [3]:
#BS4 object
html = browser.html
mars_soup = bs(html, 'html.parser')

In [4]:
headers = mars_soup.find_all('th')
info = mars_soup.find_all('tr', class_ = 'data-row')

In [5]:
#Empty lists to hold necessary info
table_headers = []
ids = []
terrestrial_dates = []
sols = []
ls = []
months = []
min_temps = []
pressures = []

#Table headers
for header in headers:
    table_headers.append(header.text)

for information in info:
   #appending id
    mars_id = information.find('td')
    ids.append(int(mars_id.text))
    
    #terrestial dates
    terrestrial_dates.append(mars_id.find_next_sibling('td').text)
    mars_id = mars_id.find_next_sibling('td')
    
    #sol
    sols.append(int(mars_id.find_next_sibling("td").text))
    mars_id = mars_id.find_next_sibling("td")
    
    #ls
    ls.append(int(mars_id.find_next_sibling("td").text))
    mars_id=mars_id.find_next_sibling('td')
    
    # month
    months.append(int(mars_id.find_next_sibling("td").text))
    mars_id = mars_id.find_next_sibling("td")
    
    # min_temp
    min_temps.append(float(mars_id.find_next_sibling("td").text))
    mars_id = mars_id.find_next_sibling("td")
    
    # pressure
    pressures.append(float(mars_id.find_next_sibling("td").text))

In [13]:
mars_df =pd.DataFrame(columns = table_headers)
mars_df["id"] = ids 
mars_df["terrestrial_date"] = terrestrial_dates
mars_df["sol"] = sols
mars_df["ls"] = ls
mars_df["month"] = months
mars_df["min_temp"] = min_temps
mars_df["pressure"] = pressures
mars_df

,id,terrestrial_date,sol,ls,month,min_temp,pressure
0,2,2012-08-16,10,155,6,-75.0,739.0
1,13,2012-08-17,11,156,6,-76.0,740.0
2,24,2012-08-18,12,156,6,-76.0,741.0
3,35,2012-08-19,13,157,6,-74.0,732.0
4,46,2012-08-20,14,157,6,-74.0,740.0
...,...,...,...,...,...,...,...
1862,1889,2018-02-23,1973,133,5,-78.0,730.0
1863,1892,2018-02-24,1974,134,5,-77.0,729.0
1864,1894,2018-02-25,1975,134,5,-76.0,729.0
1865,1893,2018-02-26,1976,135,5,-77.0,728.0


In [17]:
# Convert terrestrial_date column from string to datetime
mars_df["terrestrial_date"] = mars_df["terrestrial_date"].astype("datetime64[ns]")
mars_df

,id,terrestrial_date,sol,ls,month,min_temp,pressure
0,2,2012-08-16,10,155,6,-75.0,739.0
1,13,2012-08-17,11,156,6,-76.0,740.0
2,24,2012-08-18,12,156,6,-76.0,741.0
3,35,2012-08-19,13,157,6,-74.0,732.0
4,46,2012-08-20,14,157,6,-74.0,740.0
...,...,...,...,...,...,...,...
1862,1889,2018-02-23,1973,133,5,-78.0,730.0
1863,1892,2018-02-24,1974,134,5,-77.0,729.0
1864,1894,2018-02-25,1975,134,5,-76.0,729.0
1865,1893,2018-02-26,1976,135,5,-77.0,728.0


In [20]:
#How many months exist on Mars?
(mars_df["month"].unique())

array([ 6,  7,  8,  9, 10, 11, 12,  1,  2,  3,  4,  5], dtype=int64)